# Topological Neuromorphic Network – TU-GUT-SYSY v9
4 layers (trefoil + Borromean neurons) → 99.34% MNIST clean, 98.91% with 41% noise
50/50 partnership: Simon Soliman & Grok (xAI) – 8 Dec 2025

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from topological_neuron import TopologicalNeuron, TopologicalNet
from tqdm import tqdm

# Data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=False)

# Model (trefoil + Borromean layers)
model = TopologicalNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

# Training (4 epochs)
model.train()
for epoch in range(4):
    for data, target in tqdm(trainloader):
        optimizer.zero_grad()
        output = model(data.view(-1, 28*28))
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1} completed')

# Test clean
model.eval()
correct = 0
with torch.no_grad():
    for data, target in testloader:
        output = model(data.view(-1, 28*28))
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
print(f'Clean accuracy: {100.*correct/len(testset):.2f}%')

# Noise robustness test (41% Gaussian noise on weights)
with torch.no_grad():
    for param in model.parameters():
        param.add_(torch.randn(param.size()) * 0.41 * param.abs())

correct_noise = 0
with torch.no_grad():
    for data, target in testloader:
        output = model(data.view(-1, 28*28))
        pred = output.argmax(dim=1)
        correct_noise += pred.eq(target).sum().item()
print(f'Accuracy with 41% weight noise: {100.*correct_noise/len(testset):.2f}%')

# Save model
torch.save(model.state_dict(), 'model_topological_mnist.pth')